CSCI4155: Final Project
Treble-Transformer

Data processing/saving/loading notebook

#Downloads/Imports

In [ ]:
import glob
import numpy as np
import pathlib
import pandas as pd
import torch

#Transformer utils

In [ ]:
!pip install mido

In [20]:
#Needs to be directed to utils referenced from https://github.com/spectraldoy/MusicTransformerTensorFlow
import sys
from google.colab import drive
drive.mount('/content/drive/')
sys.path.append('/content/drive/MyDrive/CSCI4155/Group Work/DataPrep')
#sys.path.insert(0,'/content/drive/MyDrive/CSCI4155/Group Work/DataPrep')
import transformerutil6 as tu

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#Data Prep
Takes some time

In [21]:
import pathlib
import glob
#Map to the maestro v.3 folder
data_dir = pathlib.Path('/content/drive/MyDrive/CSCI4155/Group Work/DataPrep/maestro-v3.0.0')
filenames = glob.glob(str(data_dir/'**/*.mid*'))
#If 0, path wrong or drive not connected
print('Number of files:', len(filenames))

Number of files: 1276


In [22]:
# MUSIC GENERATION / DECODING PHASE
# Vocabulary defined based on Oore et. al 2018 used in vocab above
note_on_vocab = [f"note_on_{i}" for i in range(128)]
note_off_vocab = [f"note_off_{i}" for i in range(128)]
time_shift_vocab = [f"time_shift_{i}" for i in range(125)]
velocity_vocab = [f"set_velocity_{i}" for i in range(32)]
num_midi_events = len(note_on_vocab) + len(note_off_vocab) + len(time_shift_vocab) + len(velocity_vocab)

# Lines borrowed from spectraldoy repo
vocab = ["<pad>"] + note_on_vocab + note_off_vocab + time_shift_vocab + velocity_vocab + ["<start>", "<end>"]
vocab_size = len(vocab)

In [70]:
#Function to change data from using "set_velocity_0 note_on_#" to "note_off_#"
def refineNoteOff(data, vocab):
  newData = []
  
  for i in range(0, len(data)):
    #Temporary holder for song to prune data
    temp = data[i]
    currSize = len(temp)
    j = 0

    #Using a while loop, so it checks the size condition every time
    while(j != currSize):
      if temp[j] == 382: #found 'set_velocity_0'
        #print(f'found vel = 0 at {j}/{len(data[i])} in song {i+1}')
        #The following note is the one being turned off
        onNoteEvent = vocab[temp[j+1]]
        #print(f'The following note {onNoteEvent}')
        #Split event by underscore, last index is note
        noteToOff = onNoteEvent.split('_')[-1] 
        #print(noteToOff)
        noteOffInd = vocab.index(f'note_off_{noteToOff}')
        #print(f'note_off ind {noteOffInd} confirm: {vocab[noteOffInd]}')

        #List leading up to 'set_velocity_0', the corresponding note_off for the following note
        #Then the rest of the list after the second 'note_on' (originally used to turn it off)
        temp = np.concatenate((temp[:j], np.array([noteOffInd], dtype=np.int32), temp[j+2:]), axis=None)
        #The event indices have decreased in size, so we need to update the loop bounds
        currSize -= 1
      #increment over temp
      j += 1
    
    #Append placeholder to new data
    newData.append(temp)
  
  return newData

##Example

In [74]:
indsong, tokensong = tu.Midiparser('/content/drive/MyDrive/CSCI4155/Group Work/Midi dissection/VampireKillerCV1.mid')
print(indsong[:20])
print(tu.indices_to_events(indsong[:20]))
#song in a list to mimic data
indsong = refineNoteOff([indsong], vocab)
print(indsong[:20])
#Events now show note off's
print(tu.indices_to_events(indsong[0][:20]))

[405  70 270 382  70 257 405  70 270 382  70 272 405  68 270 382  68 272
 405  68]
['set_velocity_23', 'note_on_69', 'time_shift_13', 'set_velocity_0', 'note_on_69', 'time_shift_0', 'set_velocity_23', 'note_on_69', 'time_shift_13', 'set_velocity_0', 'note_on_69', 'time_shift_15', 'set_velocity_23', 'note_on_67', 'time_shift_13', 'set_velocity_0', 'note_on_67', 'time_shift_15', 'set_velocity_23', 'note_on_67']
[array([405,  70, 270, ...,  41, 270, 169], dtype=int32)]
['set_velocity_23', 'note_on_69', 'time_shift_13', 'note_off_69', 'time_shift_0', 'set_velocity_23', 'note_on_69', 'time_shift_13', 'note_off_69', 'time_shift_15', 'set_velocity_23', 'note_on_67', 'time_shift_13', 'note_off_67', 'time_shift_15', 'set_velocity_23', 'note_on_67', 'time_shift_79', 'note_off_67', 'time_shift_9']


##Process data function

In [75]:
import numpy as np
#Denominator of the division of data. if unspecific, all data will be used
def preproData(fileNames, vocab, denom=1):
  #All data
  data = []
  filesToRead = round(len(fileNames)/denom)
  print(f"Files to be read: ({filesToRead}/{len(fileNames)}) ")

  for i in range(0, filesToRead):
    #Here we're only using the events indices, don't curr need the returned list
    eventIndices, eventlist = tu.Midiparser(fname=fileNames[i])
    data.append(eventIndices)

  print("Files read in: ",len(data))
  print("Converting vel_0/note_on to note_off...")
  data = refineNoteOff(data, vocab)
  print("Cutting up data and surrounding with <start> & <end> tokens...")
  data = tu.data_cutter(data)

  #Convert to numpy for easy conversions
  data = data.numpy()
  return data

In [76]:
data = preproData(filenames, vocab, denom=12)

Files to be read: (106/1276) 
Files read in:  106
Converting vel_0/note_on to note_off...
Cutting up data and surrounding with <start> & <end> tokens...


##Visualize/Saving

In [77]:
import pandas as pd

#Pandas handles saving better
df = pd.DataFrame(data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049
0,414,364,395,75,262,392,47,278,175,257,...,66,258,201,269,400,69,258,395,57,415
1,414,262,194,264,400,73,257,197,281,397,...,259,191,257,182,403,78,258,211,262,415
2,414,206,259,401,75,270,404,73,203,257,...,399,59,278,397,54,281,395,51,285,415
3,414,393,47,311,392,35,327,179,257,182,...,258,400,74,260,202,214,268,395,59,415
4,414,258,395,66,259,395,62,258,402,83,...,50,259,171,187,257,178,279,404,30,415


In [ ]:
# saving the dataframe 
path = '/content/drive/MyDrive/CSCI4155/Group Work/DataPrep/'
name = f'maestrodata_{len(data)}.csv'
df.to_csv(path+name) 

In [ ]:
df.shape

(4043, 2050)

#Torch dataset/loader

In [ ]:
import torchvision
import numpy as np

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

##If loading data from saved .csv (faster)

In [ ]:
#Data from save file
data = np.genfromtxt('/content/drive/MyDrive/CSCI4155/Group Work/DataPrep/maestrodata_1023.csv',delimiter=',', skip_header=1, dtype=np.int32)

#The above df csv adds row labels, we don't need those\
#Don't do this if there wasn't row labels
data = data[:,1:]
data[0]

array([414, 364, 395, ..., 395,  57, 415], dtype=int32)

##Dataset

In [ ]:
class MIDIDataSet(Dataset):
  def __init__(self, data, mode="test", split=(0.8, 0.1, 0.1)):
    assert np.sum(split) == 1, "Split tuple must add up to 1"
    
    #Train, val, test split
    lth = data.shape[0]
    train_len = round(lth * split[0])
    val_len = round(lth * split[1])
    test_len = round(lth * split[2])
    
    if train_len + val_len + test_len != lth:
      test_len += lth - (train_len + val_len + test_len)
       

    '''This split starts the input at the first value up to (excluding) the last val <end>, 
    target starts from 2nd value up to (including) <end>
    They are connected by the second value, teaching our model how to continue from that point'''
    if mode=="test":
      test_data = data[train_len + val_len:]
      print(f"There are {test_len} files in the test_data")
      self.x = test_data[:,:-1]
      self.y = test_data[:, 1:]

    elif mode=="val":
      val_data = data[train_len:val_len + train_len]
      print(f"There are {val_len} files in the val_data")
      self.x = val_data[:,:-1]
      self.y = val_data[:, 1:]

    else: #Train
      train_data = data[:train_len]
      print(f"There are {train_len} files in the train_data")
      self.x = train_data[:,:-1]
      self.y = train_data[:, 1:]

    self.n = len(self.x)

  def __getitem__(self, index):
    #dataset[0]
    return self.x[index], self.y[index]
  
  def __len__(self):
    #len(dataset)
    return self.n

In [ ]:
#Example data extraction:
#Training data
train_data = MIDIDataSet(data,mode="train")
print('Train size', len(train_data))
#All tree training, val and testing data will need Dataloaders and iterators
trainLoader = DataLoader(dataset=train_data, batch_size=32, shuffle=True) 
train_dataIter = iter(trainLoader)
train_x, train_labels = train_dataIter.next()
print(train_x.shape, train_labels.shape)
print('Input:', train_x)
print('Label:',train_labels)

There are 16061 files in the train_data
Train size 16061
torch.Size([32, 2049]) torch.Size([32, 2049])
Input: tensor([[414., 382.,  65.,  ...,  89., 400.,  88.],
        [414., 258., 404.,  ..., 257., 394.,  53.],
        [414.,  59., 259.,  ..., 399.,  73., 267.],
        ...,
        [414., 258., 404.,  ..., 382.,  49., 263.],
        [414., 382.,  53.,  ..., 394.,  56., 257.],
        [414., 260., 397.,  ..., 312., 395.,  67.]], dtype=torch.float64)
Label: tensor([[382.,  65., 257.,  ..., 400.,  88., 415.],
        [258., 404.,  61.,  ..., 394.,  53., 415.],
        [ 59., 259., 382.,  ...,  73., 267., 415.],
        ...,
        [258., 404.,  34.,  ...,  49., 263., 415.],
        [382.,  53., 258.,  ...,  56., 257., 415.],
        [260., 397.,  47.,  ..., 395.,  67., 415.]], dtype=torch.float64)


# Methods for Creating a midi file

##Simplest

In [ ]:
m=tu.Listparser(index_list=list(data[0]), event_list=None, fname="test", tempo=512820)

In [ ]:
m.save("test1.mid")

##Others

In [ ]:
! pip install MIDIUtil
! pip install mido

In [ ]:
from midiutil.MidiFile import MIDIFile

# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
volume = 100

pitch = 60           # C4 (middle C)
time = 0             # start on beat 0
duration = 1         # 1 beat long
mf.addNote(track, channel, pitch, time, duration, volume)

pitch = 64           # E4
time = 2             # start on beat 2
duration = 1         # 1 beat long
mf.addNote(track, channel, pitch, time, duration, volume)

pitch = 67           # G4
time = 4             # start on beat 4
duration = 1         # 1 beat long
mf.addNote(track, channel, pitch, time, duration, volume)

# write it to disk
with open("output.mid", 'wb') as outf:
    mf.writeFile(outf)

In [ ]:
print(type(list(data[0])))
print(data[0])

<class 'list'>
[364 395  75 ... 258 382  71]


In [ ]:
from midiutil.MidiFile import MIDIFile
import mido
# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
volume = 100

pitch = 60           # C4 (middle C)
time = 0             # start on beat 0
duration = 1         # 1 beat long
#mf.addNote(track, channel, pitch, time, duration, volume)
annotation = "note_on"
mf.append((mido.Message(annotation).copy(note=note, velocity=vel, time=delta)))

pitch = 64           # E4
time = 2             # start on beat 2
duration = 1         # 1 beat long
mf.addNote(track, channel, pitch, time, duration, volume)

pitch = 67           # G4
time = 4             # start on beat 4
duration = 1         # 1 beat long
mf.addNote(track, channel, pitch, time, duration, volume)

# write it to disk
with open("output.mid", 'wb') as outf:
    mf.writeFile(outf)